In [1]:
import os

folder = "data/cleaned_txt"
files = sorted([f for f in os.listdir(folder) if f.endswith(".txt")])


In [2]:
import re
from datetime import datetime

def ekstrak_metadata(teks):
    no_perkara = re.search(r"(Nomor|No\.)\s*Perkara[:\s]*([A-Za-z0-9\/\.\-]+)", teks)
    tanggal = re.search(r"\b(\d{1,2}[\-/]\d{1,2}[\-/]\d{4})\b", teks)
    pasal = re.findall(r"Pasal\s+\d+[A-Za-z]*", teks)
    pihak = re.findall(r"(Terdakwa|Penggugat|Tergugat)\s*[:\-–]\s*([A-Z][^\n\r]+)", teks)

    return {
        "no_perkara": no_perkara.group(2) if no_perkara else "",
        "tanggal": tanggal.group(1) if tanggal else "",
        "pasal": "; ".join(set(pasal)),
        "pihak": "; ".join([f"{p[0]}: {p[1].strip()}" for p in pihak]),
    }


In [3]:
def ekstrak_fakta(teks):
    match = re.search(r"(menimbang\s+bahwa.*?)((menimbang\s+bahwa|menurut\s+hukum|amar)|$)", teks, re.IGNORECASE | re.DOTALL)
    return match.group(1).strip() if match else ""

def ekstrak_argumen(teks):
    match = re.search(r"(menurut\s+hukum.*?)(amar\s+putusan|demikian|putusan)", teks, re.IGNORECASE | re.DOTALL)
    return match.group(1).strip() if match else ""


In [4]:
def fitur_teks(teks):
    return {
        "length": len(teks.split()),  # jumlah kata
        "has_pasal": "pasal" in teks.lower(),
    }


In [5]:
import pandas as pd

data = []
for i, file in enumerate(files):
    with open(os.path.join(folder, file), "r", encoding="utf-8") as f:
        teks = f.read()

    meta = ekstrak_metadata(teks)
    fakta = ekstrak_fakta(teks)
    argumen = ekstrak_argumen(teks)
    fitur = fitur_teks(teks)

    data.append({
        "case_id": i + 1,
        "filename": file,
        "no_perkara": meta["no_perkara"],
        "tanggal": meta["tanggal"],
        "pasal": meta["pasal"],
        "pihak": meta["pihak"],
        "ringkasan_fakta": fakta[:500],  # potong biar ringkas
        "argumen_hukum": argumen[:500],
        "text_full": teks[:1000],
        "length": fitur["length"],
        "has_pasal": fitur["has_pasal"],
    })

# Simpan ke CSV dan JSON
os.makedirs("data/processed", exist_ok=True)
pd.DataFrame(data).to_csv("data/processed/cases.csv", index=False)
pd.DataFrame(data).to_json("data/processed/cases.json", orient="records", indent=2)


In [6]:
import os
import re
import pandas as pd

folder_txt = "data/cleaned_txt"
files = sorted([f for f in os.listdir(folder_txt) if f.endswith(".txt")])

def ekstrak_metadata(teks):
    # Nomor perkara
    no_perkara = re.search(r"(nomor|no\.)\s*perkara[:\s]*([A-Za-z0-9\/\.\-]+)", teks, re.IGNORECASE)
    
    # Tanggal
    tanggal = re.search(r"\b(\d{1,2}[-/]\d{1,2}[-/]\d{4})\b", teks)
    
    # Pasal
    pasal = re.findall(r"pasal\s+\d+[A-Za-z]*\s*(?:ayat\s*\([^)]+\))?", teks, re.IGNORECASE)
    
    # Nama pihak (penggugat/tergugat/terdakwa)
    pihak = re.findall(r"(terdakwa|penggugat|tergugat)[:\-–]?\s*([A-Z][^\n\r]+)", teks, re.IGNORECASE)
    
    return {
        "no_perkara": no_perkara.group(2) if no_perkara else "",
        "tanggal": tanggal.group(1) if tanggal else "",
        "pasal": "; ".join(set(pasal)),
        "pihak": "; ".join([f"{p[0].capitalize()}: {p[1].strip()}" for p in pihak])
    }

def ekstrak_ringkasan_fakta(teks):
    match = re.search(r"(menimbang\s+bahwa.*?)(menimbang|mengingat|amar)", teks, re.IGNORECASE | re.DOTALL)
    return match.group(1).strip() if match else teks[:700]

# Gabungkan semuanya
case_data = []

for idx, file in enumerate(files):
    with open(os.path.join(folder_txt, file), "r", encoding="utf-8") as f:
        teks = f.read()
    
    metadata = ekstrak_metadata(teks)
    ringkasan = ekstrak_ringkasan_fakta(teks)
    
    case_data.append({
        "case_id": idx + 1,
        "no_perkara": metadata["no_perkara"],
        "tanggal": metadata["tanggal"],
        "ringkasan_fakta": ringkasan,
        "pasal": metadata["pasal"],
        "pihak": metadata["pihak"],
        "text_full": teks[:3000]  # atau simpan seluruh teks
    })

# Simpan ke CSV
os.makedirs("data/processed", exist_ok=True)
df = pd.DataFrame(case_data)
df.to_csv("data/processed/cases.csv", index=False, encoding="utf-8-sig")


In [7]:
for f in os.listdir("data/cleaned_txt"):
    with open(os.path.join("data/cleaned_txt", f), encoding="utf-8") as file:
        isi = file.read()
        print(f"\n==== {f} ====\n", isi[:1000])  # tampilkan awal isi
        break  # hanya 1 file untuk lihat



==== case (1).txt ====
 direktori putusan mahkamah agung republik indonesia putusanmahkamahagunggoid mahkamah agung republik indonesia mahkamah agung republik indonesia mahkamah agung republik indonesia mahkamah agung republik indonesia mahkamah agung republik indonesia disclaimer kepaniteraan mahkamah agung republik indonesia berusaha mencantumkan informasi akurat bentuk komitmen mahkamah agung pelayanan publik transparansi akuntabilitas pelaksanaan fungsi peradilan halhal permasalahan teknis terkait akurasi keterkinian informasi sajikan perbaiki kewaktu menemukan inakurasi informasi termuat situs informasi tersedia harap hubungi kepaniteraan mahkamah agung email kepaniteraanmahkamahagunggoid telp ext halaman putusan nomor kpidsus nomor kpidsus keadilan berdasarkan ketuhanan maha esa memeriksa perkara tindak pidana khusus tingkat kasasi dimohonkan penuntut kejaksaan negeri jakarta selatan terdakwa memutus perkara terdakwa nama fadlan djakfar lahir aceh umurtanggal lahir april jenis k

In [9]:
import pdfplumber

def ambil_header_pdf(path_pdf):
    with pdfplumber.open(path_pdf) as pdf:
        first_page = pdf.pages[0]
        teks = first_page.extract_text()
        return teks


ModuleNotFoundError: No module named 'pdfplumber'